# Features

This notebook is supposed to showcase some of the various features supported by the widget. 

For a detailed description for the different mappings, check out the widget [documentation](https://yworks.github.io/yfiles-jupyter-graphs/)


In [1]:
%pip install yfiles_jupyter_graphs_for_neo4j --quiet
%pip install neo4j --quiet
from yfiles_jupyter_graphs_for_neo4j import Neo4jGraphWidget
from neo4j import GraphDatabase

NEO4J_URI      = "neo4j+ssc://demo.neo4jlabs.com" 
NEO4J_USERNAME = "fincen"
NEO4J_PASSWORD = "fincen"
driver = GraphDatabase.driver(uri = NEO4J_URI, auth = (NEO4J_USERNAME, NEO4J_PASSWORD), database = 'fincen')

g = Neo4jGraphWidget(driver)

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


## Heat Mapping

In [2]:
max_amount = 120000000
min_amount = 50000

def heat_mapping(element):
    if 'amount' in element['properties']:
        amount = element['properties']['amount']
        normalized_value = (amount - min_amount) / (max_amount - min_amount)
        transformed_value = -1 * (1 - normalized_value) ** 2 + 1
        return max(0, min(1, transformed_value))
        
    

g.add_node_configuration('Filing', heat=heat_mapping)


g.show_cypher("MATCH (s)-[r]->(t) RETURN s,r,t LIMIT 25")

GraphWidget(layout=Layout(height='690px', width='100%'))

## Leaflet Integration

In [3]:
def node_coordinate_mapping(node):
    return (node['properties']['location']['y'], node['properties']['location']['x']) if 'location' in node['properties'].keys() else None

def filing_coordinate_mapping(node):
    return (node['properties']['beneficiary_lat'], (node['properties']['beneficiary_lng'])) if 'beneficiary_lat' in node['properties'].keys() else None

g.add_node_configuration('Country', coordinate=node_coordinate_mapping)
g.add_node_configuration('Entity', coordinate=node_coordinate_mapping)
g.add_node_configuration('Filing', coordinate=filing_coordinate_mapping)

g.show_cypher("MATCH (s)-[r]->(t) RETURN s,r,t LIMIT 50")

GraphWidget(layout=Layout(height='800px', width='100%'))

## Node Grouping

In [12]:
def node_grouping_mapping(node):
    if 'country' in node['properties'].keys():
        match node['properties']['country']:
            case 'KHM': return 330
            case 'NZL': return 454
            case 'CHN': return 338
            case 'CYP': return 350
            case 'SGP': return  497

g.add_node_configuration('Entity', parent=node_grouping_mapping)



g.show_cypher("MATCH (s)-[r]->(t) RETURN s,r,t LIMIT 22")

GraphWidget(layout=Layout(height='660px', width='100%'))

## Node Size & Edge Thickness Mapping

In [27]:

g.add_relationship_configuration('CONCERNS', thickness_factor= 0.5)
g.add_relationship_configuration('FILED', thickness_factor= 2)
g.del_node_configuration('Entity')
g.add_node_configuration('Filing', size= lambda node:  (55 * (1 + heat_mapping(node)), 55 * (1 + heat_mapping(node)))) 

g.show_cypher("MATCH (s)-[r]->(t) RETURN s,r,t LIMIT 25")

GraphWidget(layout=Layout(height='690px', width='100%'))